# Programming Assignment: Random forest

In this assignment we will not only build a **random forest**, but also try to select the number of the trees to fit a particular criteria of quality.    
Let's take a look at original six tasks presented in Russian.

**Tasks will be duplicated in english further in this assignment**

1.    Загрузите данные из файла abalone.csv. Это датасет, в котором требуется предсказать возраст ракушки (число колец) по физическим измерениям.
2. Преобразуйте признак Sex в числовой: значение F должно перейти в -1, I — в 0, M — в 1. Если вы используете Pandas, то подойдет следующий код: data['Sex'] = data['Sex'].map(lambda x: 1 if x == 'M' else (-1 if x == 'F' else 0))
3. Разделите содержимое файлов на признаки и целевую переменную. В последнем столбце записана целевая переменная, в остальных — признаки.
4. Обучите случайный лес (sklearn.ensemble.RandomForestRegressor) с различным числом деревьев: от 1 до 50 (не забудьте выставить "random_state=1" в конструкторе). Для каждого из вариантов оцените качество работы полученного леса на кросс-валидации по 5 блокам. Используйте параметры "random_state=1" и "shuffle=True" при создании генератора кросс-валидации sklearn.cross_validation.KFold.  В качестве меры качества воспользуйтесь коэффициентом детерминации (sklearn.metrics.r2_score).
5. Определите, при каком минимальном количестве деревьев случайный лес показывает качество на кросс-валидации выше 0.52. Это количество и будет ответом на задание.
6. Обратите внимание на изменение качества по мере роста числа деревьев. Ухудшается ли оно?

In [1]:
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold, cross_val_score

import pandas

## Task №1
*Ru*:  Загрузите данные из файла abalone.csv. Это датасет, в котором требуется предсказать возраст ракушки (число колец) по физическим измерениям.

*En:* Load data from abalone.csv file. This is a dataset, you want to predict the age of the shell (number of rings) from physical measurements with it.

In [2]:
df = pandas.read_csv('abalone.csv')
df.head()

,Sex,Length,Diameter,Height,WholeWeight,ShuckedWeight,VisceraWeight,ShellWeight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


## Task №2
*Ru*: Преобразуйте признак Sex в числовой: значение F должно перейти в -1, I — в 0, M — в 1. Если вы используете Pandas, то подойдет следующий код: data['Sex'] = data['Sex'].map(lambda x: 1 if x == 'M' else (-1 if x == 'F' else 0))

*En:* Convert 'Sex' feature to numeric: F should go to -1, I to 0, M to 1. If you are using Pandas use the following code: data ['Sex'] = data ['Sex']. Map ( lambda x: 1 if x == 'M' else (-1 if x == 'F' else 0)) 

In [3]:
df['Sex'] = df['Sex'].map(lambda x: 1 if x == 'M' else (-1 if x =='F' else 0))
df.head()

,Sex,Length,Diameter,Height,WholeWeight,ShuckedWeight,VisceraWeight,ShellWeight,Rings
0,1,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,-1,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,1,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,0,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


## Task №3
*Ru*: Разделите содержимое файлов на признаки и целевую переменную. В последнем столбце записана целевая переменная, в остальных — признаки.

*En:* Split the data in the file into feature and target variable. The last column contains the target variable, the rest - features. 

In [4]:
X = df.loc[:,'Sex':'ShellWeight']
X.head()

,Sex,Length,Diameter,Height,WholeWeight,ShuckedWeight,VisceraWeight,ShellWeight
0,1,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150
1,1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070
2,-1,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210
3,1,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155
4,0,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055


In [5]:
y = df['Rings']
y

0       15
1        7
2        9
3       10
4        7
        ..
4172    11
4173    10
4174     9
4175    10
4176    12
Name: Rings, Length: 4177, dtype: int64

## Task №4
*Ru*: Обучите случайный лес (sklearn.ensemble.RandomForestRegressor) с различным числом деревьев: от 1 до 50 (не забудьте выставить "random_state=1" в конструкторе). Для каждого из вариантов оцените качество работы полученного леса на кросс-валидации по 5 блокам. Используйте параметры "random_state=1" и "shuffle=True" при создании генератора кросс-валидации sklearn.cross_validation.KFold.  В качестве меры качества воспользуйтесь коэффициентом детерминации (sklearn.metrics.r2_score).


*En:* Train a random forest (sklearn.ensemble.RandomForestRegressor) with a different number of trees: from 1 to 50 (don't forget to set "random_state = 1" in the constructor). For each of the options, evaluate the quality of the work of the resulting forest on cross-validation for 5 blocks. Use the "random_state = 1" and "shuffle = True" parameters when creating the sklearn.cross_validation.KFold cross-validation generator. Use the coefficient of determination (sklearn.metrics.r2_score) as a measure of quality. 

In [6]:
cv = KFold(random_state=1, shuffle=True)
scores = []
found = False
for i in range(1, 51):
    RFR = RandomForestRegressor(random_state=1, n_estimators=i)
    RFR.fit(X, y)
    scores.append(cross_val_score(RFR, X, y, cv=cv, scoring='r2'))
    
    if np.mean(scores[i - 1]) > 0.52 and not found:
        print("number of trees should be T =", i)
        found = True

number of trees should be T = 21


## Task №5
*Ru*: Определите, при каком минимальном количестве деревьев случайный лес показывает качество на кросс-валидации выше 0.52. Это количество и будет ответом на задание.

*En:* Determine at what minimum number of trees the random forest shows quality on cross-validation above 0.52. This amount will be the answer to the task. 

I have already figured that out while doing previous task, but let's do it another way

In [7]:
df_scores = pandas.DataFrame(scores)
df_scores.head()

,0,1,2,3,4
0,0.196642,0.117624,0.250149,0.017945,-0.033987
1,0.349990,0.312488,0.414424,0.293152,0.336446
2,0.435556,0.394510,0.450715,0.343694,0.407694
3,0.453891,0.431817,0.488333,0.401717,0.448115
4,0.465166,0.461835,0.502452,0.424564,0.471144


In [8]:
df_scores = df_scores.mean(1)

In [9]:
df_scores = df_scores.map(lambda x: x if x > 0.52 else None)
df_scores

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
5          NaN
6          NaN
7          NaN
8          NaN
9          NaN
10         NaN
11         NaN
12         NaN
13         NaN
14         NaN
15         NaN
16         NaN
17         NaN
18         NaN
19         NaN
20    0.520529
21    0.520804
22    0.521743
23    0.523106
24    0.523249
25    0.524308
26    0.524639
27    0.525656
28    0.526556
29    0.527086
30    0.527642
31    0.528924
32    0.530107
33    0.529961
34    0.529821
35    0.529910
36    0.529432
37    0.529472
38    0.529516
39    0.529470
40    0.529809
41    0.530043
42    0.529914
43    0.529681
44    0.528908
45    0.529008
46    0.529179
47    0.530051
48    0.530813
49    0.530951
dtype: float64

We got 21 once again

## Task №6
*Ru*: Обратите внимание на изменение качества по мере роста числа деревьев. Ухудшается ли оно?

*En:* Notice how the quality changes as the number of trees grows. Is it getting worse? 

**Answer:** Nope, it's only get better and better: random forest don't suffer from overfitting that much